In [1]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [2]:
# t0 = time.time()

# topic_name = 'test-topic'

# tsend_start = time.time()
# for i in range(10):
#     message = {'number': i}
#     producer.send(topic_name, value=message)
#     print(f"Sent: {message}")
#     time.sleep(0.05)
# tsend_end = time.time()

# tflush_start = time.time()
# producer.flush()
# tflush_end = time.time()

# t1 = time.time()
# print(f'took {(t1 - t0):.2f} seconds')
# print(f'{(tsend_end-tsend_start)} seconds sending time.')
# print(f'{(tflush_end-tflush_start)} seconds flushing time.')

In [2]:
import pandas as pd

def csv_reader(url : str) -> pd.DataFrame :
    iterator = pd.read_csv(url, iterator=True, chunksize=100000, sep=',', compression='gzip')
    df = pd.DataFrame()
    try:
        while True:
            data = next(iterator)
            if len(df)==0:
                df=data
            else:
                df=pd.concat([df,data])
            
    except StopIteration:
        print('No next row.')

    return df

def load_data_from_api(*args, **kwargs):
    """
    Template for loading data from API
    """
    # data_types = {
    #     'VendorID': pd.Int64Dtype(),
    #     'store_and_fwd_flag': str,
    #     'RatecodeID': pd.Int64Dtype(),
    #     'PULocationID': pd.Int64Dtype(),
    #     'DOLocationID': pd.Int64Dtype(),
    #     'passenger_count': pd.Int64Dtype(),
    #     'trip_distance': float,
    #     'fare_amount': float,
    #     'extra': float,
    #     'mta_tax': float,
    #     'tip_amount': float,
    #     'tolls_amount': float,
    #     'ehail_fee': float,
    #     'improvement_surcharge': float,
    #     'total_amount': float,
    #     'payment_type': pd.Int64Dtype(),
    #     'trip_type': pd.Int64Dtype(),
    #     'congestion_surcharge': float
    # }
    # parse_dates = ['lpep_pickup_datetime','lpep_dropoff_datetime']

    url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz'
    frame = csv_reader(url)
            
                
    return frame

df = load_data_from_api()

No next row.


/tmp/ipykernel_59427/571853930.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = next(iterator)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               387007 non-null  float64
 1   lpep_pickup_datetime   476386 non-null  object 
 2   lpep_dropoff_datetime  476386 non-null  object 
 3   store_and_fwd_flag     387007 non-null  object 
 4   RatecodeID             387007 non-null  float64
 5   PULocationID           476386 non-null  int64  
 6   DOLocationID           476386 non-null  int64  
 7   passenger_count        387007 non-null  float64
 8   trip_distance          476386 non-null  float64
 9   fare_amount            476386 non-null  float64
 10  extra                  476386 non-null  float64
 11  mta_tax                476386 non-null  float64
 12  tip_amount             476386 non-null  float64
 13  tolls_amount           476386 non-null  float64
 14  ehail_fee              0 non-null   

In [4]:
main_df = df[[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
    ]]

topic_name = 'green-trips'
tstart = time.time()
tsend_start = time.time()
for row in main_df.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)
tsend_end = time.time()

tflush_start = time.time()
producer.flush()
tflush_end = time.time()

tend = time.time()
print(f'took {(tend - tstart):.2f} seconds')
print(f'{(tsend_end-tsend_start)} seconds sending time.')
print(f'{(tflush_end-tflush_start)} seconds flushing time.')

took 44.52 seconds
40.28867030143738 seconds sending time.
4.2281694412231445 seconds flushing time.


In [5]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/04/09 15:45:16 WARN Utils: Your hostname, LAPTOP-P1QAHOCG resolves to a loopback address: 127.0.1.1; using 172.26.185.65 instead (on interface eth0)
24/04/09 15:45:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/alexxgo21/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/alexxgo21/.ivy2/cache
The jars for the packages stored in: /home/alexxgo21/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5cd26623-c57c-48b2-8a0c-553cbbfa6155;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:

24/04/09 15:45:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [7]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row)

query = green_stream.writeStream.foreachBatch(peek).start()

24/04/09 15:45:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5061116f-b0ce-4427-b09a-d34c48c132c0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 15:45:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


[Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 4, 9, 15, 44, 30, 635000), timestampType=0)]


In [8]:
query.stop()

In [9]:
from pyspark.sql import types

schema = types.StructType() \
    .add('lpep_pickup_datetime', types.StringType()) \
    .add('lpep_dropoff_datetime', types.StringType()) \
    .add('PULocationID', types.IntegerType()) \
    .add('DOLocationID', types.IntegerType()) \
    .add('passenger_count', types.DoubleType()) \
    .add('trip_distance', types.DoubleType()) \
    .add('tip_amount', types.DoubleType())

In [10]:
from pyspark.sql import functions as F

def debug_and_process(batch_df, batch_id):
    batch_df.select(F.col("value").cast('STRING')).show(truncate=False)
    parsed_batch = batch_df.select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
    .select("data.*")
    
    print(f"Batch ID: {batch_id}")
    parsed_batch.show(truncate=False)

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [11]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/04/09 15:45:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-73672a6f-2700-4d4e-907f-9150281bb2ff. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/09 15:45:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


[Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)]


In [12]:
query.stop()

In [13]:
from pyspark.sql.functions import current_timestamp, window, count
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.getOrCreate()

# Read the green csv file
df_green = spark.read.option("header", True).csv("./green_tripdata_2019-10.csv")

# Add the timestamp column
df_green_with_timestamp = df_green.withColumn("timestamp", current_timestamp())

# Define a window specification
window_spec = window(df_green_with_timestamp["timestamp"], "5 minutes")

# Group by the window specification and "DOLocationID", then count the number of occurrences in each group
windowed_df = df_green_with_timestamp.groupBy(
    window_spec,
    df_green_with_timestamp["DOLocationID"]
).count()

# Order by count in descending order
windowed_df = windowed_df.orderBy(windowed_df["count"].desc())

# Show the result
windowed_df.show()

+--------------------+------------+-----+
|              window|DOLocationID|count|
+--------------------+------------+-----+
|{2024-04-09 15:45...|          74|17741|
|{2024-04-09 15:45...|          42|15942|
|{2024-04-09 15:45...|          41|14061|
|{2024-04-09 15:45...|          75|12840|
|{2024-04-09 15:45...|         129|11930|
|{2024-04-09 15:45...|           7|11533|
|{2024-04-09 15:45...|         166|10845|
|{2024-04-09 15:45...|         236| 7913|
|{2024-04-09 15:45...|         223| 7542|
|{2024-04-09 15:45...|         238| 7318|
|{2024-04-09 15:45...|          82| 7292|
|{2024-04-09 15:45...|         181| 7282|
|{2024-04-09 15:45...|          95| 7244|
|{2024-04-09 15:45...|         244| 6733|
|{2024-04-09 15:45...|          61| 6606|
|{2024-04-09 15:45...|         116| 6339|
|{2024-04-09 15:45...|         138| 6144|
|{2024-04-09 15:45...|          97| 6050|
|{2024-04-09 15:45...|          49| 5221|
|{2024-04-09 15:45...|         151| 5153|
+--------------------+------------